In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns

# Assign Engagement and Experience Scores
We will compute the engagement score and experience score using the Euclidean distance between each user and the less engaged/worst experience cluster (from the k-means clustering results):

In [ ]:
# Engagement score (distance from less engaged cluster)
less_engaged_cluster_center = cluster_centers_[0]  # replace with the actual less engaged cluster
engagement_score = pairwise_distances(user_data, [less_engaged_cluster_center], metric='euclidean').flatten()


In [ ]:
# Experience score (distance from worst experience cluster)
worst_experience_cluster_center = cluster_centers_[1]  # replace with the actual worst experience cluster
experience_score = pairwise_distances(user_data, [worst_experience_cluster_center], metric='euclidean').flatten()
